In [1]:
from envVMWM import *
exe_location='C:\\Users\\YuHang\\Desktop\\Water_Maze\\v0.17\\VMWM.exe'
cfg_location = 'C:\\Users\\YuHang\\Desktop\\Water_Maze\\v0.17\\VMWM_data\\configuration_original.txt'

In [2]:
env = VMWMGame(cfg_location)
env.reset_cfg()

env.set_trial('Practice - Hills')
env.set_exe_location(exe_location)
env.set_IP_address('127.0.0.1')

In [ ]:
env.start()
env.start_trial()

Sleep for 10s to ensure fully load up.
connection start
connection established
Done


In [ ]:
import cv2
cv2.startWindowThread()

for i in range(20):
    while(not env.is_episode_finished()):
        cv2.imshow('frame', env.get_screenImage())
        cv2.waitKey(1)
        env.make_action(2,50)
        #print(env.get_reward())
        #print(env.get_score())
    env.new_episode()
    
env.end_trial()
env.s.close()
cv2.destroyAllWindows()